In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
from tqdm.notebook import tqdm
# from tqdm import tqdm


In [2]:
# only disable warnings when you're sure you want to do it
import warnings
warnings.filterwarnings("ignore")

In [3]:
# method for en document
# target_url
# URL = "https://www.gesetze-im-internet.de/Teilliste_translations.html"
# ENDPOINT = "https://www.gesetze-im-internet.de/" 
# set target folder
DATA_FOLDER = "./data/de"

In [4]:
# request = requests.request(url=URL, method="POST")
# soup = BeautifulSoup(request.text, 'html.parser')
# # Find all <a> tags
# anchor_tags = soup.find_all('a')

In [5]:
# # the first 15 links are not relevant
# links = pd.DataFrame(anchor_tags, columns=["anchor_tags"])
# # filter after the dataframe correctly recognized the it's anchor tag
# links = links.iloc[15:-6].reset_index(drop=True)
# # get links
# links.loc[:, "link"] = links.loc[:, "anchor_tags"].apply(lambda x:x.get("href"))
# links.loc[:, "description"] = links.loc[:, "anchor_tags"].apply(lambda x:x.abbr.get("title"))
# # we want to go to englisch_abgg/englisch_abgg.html
# links.loc[:, "full_link"] = links.loc[:, "link"].apply(lambda x:ENDPOINT + x.split("/")[0] + "/" + x.split("/")[0] + ".html")
# links.head()

In [45]:
def html_extration_en(base_url: pd.DataFrame) -> dict:
    """
    This function will extract certain inforamtion from the gesetze website.
    The function will return a list of dictionary as follows: 
    [
    {"section": first section detail,
    "content": law content,
    "link": link to the specific paragraph(by id)},
    {"section": second section detail,
    "content": law content,
    "link": link to the specific paragraph(by id)}
    ]
    For example:
    Given the url: 
    https://www.gesetze-im-internet.de/englisch_abgg/englisch_abgg.html

    the element could be:

    <p style="text-align: center; font-weight: bold"><a name="p0017"><!----></a>Section 2<br>Protection of the free exercise of an electoral mandate</p>
    <p><a name="p0018"><!----></a>(1) No one may be prevented from standing as a candidate for a mandate to serve as a Member of the Bundestag or from acquiring, accepting or holding such a mandate.</p>
    <p><a name="p0019"><!----></a>(2) Discrimination at work on the grounds of candidature for or acquisition, acceptance and exercise of a mandate shall be inadmissible.</p>
    
    The first paragraph is section and section detail.
    The second paragraph is the law content.
    The third paragraph is the law content of the same section of second paragraph.

    And the function will return:
    [
    {'section': 'Section 2:Protection of the free exercise of an electoral mandate',
    'content': '(1) No one may be prevented from standing as a candidate for a mandate to serve as a Member of the Bundestag or from acquiring, accepting or holding such a mandate. (2) Discrimination at work on the grounds of candidature for or acquisition, acceptance and exercise of a mandate shall be inadmissible. (3) Termination of an employment contract or dismissal on grounds of the acquisition, acceptance or exercise of a mandate shall be inadmissible. In all other respects, termination of an employment contract shall only be permitted for a compelling reason. Protection against termination or dismissal shall take effect on the selection of the candidate by the relevant party organ or on submission of the list of nominated candidates. It shall continue to apply for one year after the end of the Member’s term of office.',
    'link': 'https://www.gesetze-im-internet.de/englisch_abgg/englisch_abgg.html#p0017'},
    ]
    """
    request = requests.request(url=base_url, method="GET")
    soup = BeautifulSoup(request.text, 'html.parser')
    # find p tags
    paragraphs = soup.find_all('p')

    # init
    current_section = None
    sections = []  
    # analyse html and get the actual content
    for p in paragraphs:
        a_tag = p.find('a')
        if a_tag and 'name' in a_tag.attrs:
            anchor_id = a_tag['name']
            # find the section by bold style
            if 'text-align: center' in p.get('style', '') and 'font-weight: bold' in p.get('style', ''):
                # replace br to make format clean
                for br in p.find_all('br'):
                    br.replace_with(':')
                # make sure the content update after changing section
                new_section = p.get_text(strip=True)
                if current_section:
                    sections.append({
                        'section': current_section,
                        'content': current_content.strip(),
                        'link': f"{base_url}#{current_anchor_id}"
                    })
                current_section = new_section
                current_content = ""
                # anchor id for creating link
                current_anchor_id = anchor_id
            else:
                # if section does not change, concatenate the content
                if current_section:
                    content = p.get_text(strip=True).replace("table of contents", "")
                    if content:
                        current_content += " " + content
    # add the lastest section back
    if current_section:
        sections.append({
            'section': current_section,
            'content': current_content.strip(),
            'link': f"{base_url}#{current_anchor_id}"
        })
    return sections

In [58]:
def html_extration_de(base_url: pd.DataFrame) -> dict:
    """
    This function will extract certain inforamtion from the gesetze website.
    The function will return a list of dictionary as follows: 
    [
    {"section": first section detail,
    "content": law content,
    "link": link to the specific paragraph(by id)},
    {"section": second section detail,
    "content": law content,
    "link": link to the specific paragraph(by id)}
    ]
    For example:
    Given the url: 
    https://www.gesetze-im-internet.de/englisch_abgg/englisch_abgg.html

    the element could be:

    <p style="text-align: center; font-weight: bold"><a name="p0017"><!----></a>Section 2<br>Protection of the free exercise of an electoral mandate</p>
    <p><a name="p0018"><!----></a>(1) No one may be prevented from standing as a candidate for a mandate to serve as a Member of the Bundestag or from acquiring, accepting or holding such a mandate.</p>
    <p><a name="p0019"><!----></a>(2) Discrimination at work on the grounds of candidature for or acquisition, acceptance and exercise of a mandate shall be inadmissible.</p>
    
    The first paragraph is section and section detail.
    The second paragraph is the law content.
    The third paragraph is the law content of the same section of second paragraph.

    And the function will return:
    [
    {'section': 'Section 2:Protection of the free exercise of an electoral mandate',
    'content': '(1) No one may be prevented from standing as a candidate for a mandate to serve as a Member of the Bundestag or from acquiring, accepting or holding such a mandate. (2) Discrimination at work on the grounds of candidature for or acquisition, acceptance and exercise of a mandate shall be inadmissible. (3) Termination of an employment contract or dismissal on grounds of the acquisition, acceptance or exercise of a mandate shall be inadmissible. In all other respects, termination of an employment contract shall only be permitted for a compelling reason. Protection against termination or dismissal shall take effect on the selection of the candidate by the relevant party organ or on submission of the list of nominated candidates. It shall continue to apply for one year after the end of the Member’s term of office.',
    'link': 'https://www.gesetze-im-internet.de/englisch_abgg/englisch_abgg.html#p0017'},
    ]
    """

    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 尋找所有包含 "jnnorm" class 的 div 標籤，這裡是每個法條的主要容器
    norms = soup.find_all('div', class_='jnnorm')

    # 初始化變數
    sections = []

    # 分析 HTML 並提取標題與內容
    for norm in norms:
        current_section = None
        current_content = ""
        # 提取標題，標題由 <h3> 標籤中的內容組成
        title_tag = norm.find('h3')
        if title_tag:
            title_span1 = title_tag.find('span', class_='jnenbez')  # 取得 "Art" 或其他標題部分
            title_span2 = title_tag.find('span', class_='jnentitel')  # 取得標題名稱
            current_section = (title_span1.get_text(strip=True) if title_span1 else '') + ' ' + (title_span2.get_text(strip=True) if title_span2 else '')

        # 提取內容，內容位於 "jurAbsatz" class 的 div 中
        content_divs = norm.find_all('div', class_='jurAbsatz')
        current_content = " ".join([div.get_text(strip=True) for div in content_divs])

        # 取得 anchor ID 來生成連結
        anchor_tag = norm.find('a', {'name': True})
        if anchor_tag:
            anchor_id = anchor_tag['name']
            link = f"{base_url}index.html#{anchor_id}"
        else:
            link = None

        # 將結果加入 sections 列表
        sections.append({
            'section': current_section,
            'content': current_content.strip(),
            'link': link
        })


    return sections

In [15]:
import time
def get_doc_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # 查找 <abbr> 標籤內含有 "HTML" 的 <a> 標籤
    a_tag = soup.find('a', text="HTML")
    time.sleep(0.1)
    if a_tag:
        href = a_tag['href']
        return href
    else:
        return None

In [16]:
letter = "A"

In [27]:
URL = f"https://www.gesetze-im-internet.de/Teilliste_{letter}.html"
ENDPOINT = "https://www.gesetze-im-internet.de/" 
DATA_FOLDER = "./data"
request = requests.request(url=URL, method="POST")
soup = BeautifulSoup(request.text, 'html.parser')
anchor_tags = soup.find_all('a')
# the first 15 links are not relevant
links = pd.DataFrame(anchor_tags, columns=["anchor_tags"])
# # filter after the dataframe correctly recognized the it's anchor tag
links = links.iloc[15:-6].reset_index(drop=True)
# # get links
print("Getting link.")
links.loc[:, "link"] = links.loc[:, "anchor_tags"].apply(lambda x:x.get("href"))
links.loc[:, "description"] = links.loc[:, "anchor_tags"].apply(lambda x:x.abbr.get("title"))
print("Processing to correct format.")
# remove pdf
links = links[~links['link'].str.endswith('.pdf')].reset_index(drop=True)
# we want to go to englisch_abgg/englisch_abgg.html
links.loc[:, "full_link"] = links.loc[:, "link"].apply(lambda x:ENDPOINT + x.split("/")[1] + "/" +  get_doc_url(x.replace("./", ENDPOINT)))
# Find all <a> tags
links

Getting link.
Processing to correct format.


,anchor_tags,link,description,full_link
0,[[ AABG ]],./aabg/index.html,Gesetz zur Begrenzung der Arzneimittelausgaben...,https://www.gesetze-im-internet.de/aabg/BJNR06...
1,[[ AABGebV ]],./aabgebv/index.html,Besondere Gebührenverordnung des Auswärtigen A...,https://www.gesetze-im-internet.de/aabgebv/BJN...
2,[[ AAG ]],./aufag/index.html,Gesetz über den Ausgleich der Arbeitgeberaufwe...,https://www.gesetze-im-internet.de/aufag/BJNR3...
3,[[ AAppO ]],./aappo/index.html,Approbationsordnung für Apotheker,https://www.gesetze-im-internet.de/aappo/BJNR0...
4,[[ AarhusÜbk ]],./aarhus_bk/index.html,Übereinkommen über den Zugang zu Informationen...,https://www.gesetze-im-internet.de/aarhus_bk/B...
...,...,...,...,...
420,[[ AZAV ]],./azav/index.html,Verordnung über die Voraussetzungen und das Ve...,https://www.gesetze-im-internet.de/azav/BJNR05...
421,[[ AZG ]],./azg/index.html,Anteilzollgesetz,https://www.gesetze-im-internet.de/azg/BJNR010...
422,[[ AZRG ]],./azrg/index.html,Gesetz über das Ausländerzentralregister,https://www.gesetze-im-internet.de/azrg/BJNR22...
423,[[ AZRG-DV ]],./azrg-dv/index.html,Verordnung zur Durchführung des Gesetzes über ...,https://www.gesetze-im-internet.de/azrg-dv/BJN...


In [61]:
target_list = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", 
               "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", 
               "1", "2", "3", "4", "5", "6", "7", "8", "9"]
for letter in target_list:
    URL = f"https://www.gesetze-im-internet.de/Teilliste_{letter}.html"
    ENDPOINT = "https://www.gesetze-im-internet.de/" 
    DATA_FOLDER = "./data"
    request = requests.request(url=URL, method="POST")
    soup = BeautifulSoup(request.text, 'html.parser')
    anchor_tags = soup.find_all('a')
    # the first 15 links are not relevant
    links = pd.DataFrame(anchor_tags, columns=["anchor_tags"])
    # # filter after the dataframe correctly recognized the it's anchor tag
    links = links.iloc[15:-6].reset_index(drop=True)
    # # get links
    print(f"Getting link from subset {letter}.")
    links.loc[:, "link"] = links.loc[:, "anchor_tags"].apply(lambda x:x.get("href"))
    links.loc[:, "description"] = links.loc[:, "anchor_tags"].apply(lambda x:x.abbr.get("title"))
    print("Processing to correct format.")
    # remove pdf
    links = links[~links['link'].str.endswith('.pdf')].reset_index(drop=True)
    # we want to go to englisch_abgg/englisch_abgg.html
    links.loc[:, "full_link"] = links.loc[:, "link"].apply(lambda x:ENDPOINT + x.split("/")[1] + "/" +  get_doc_url(x.replace("./", ENDPOINT)))
    for index, row in tqdm(links.iterrows(), total=links.shape[0], leave=True, desc="Processing"):
        # item = row[1]
        # get tartget url and overall description of the law
        title = row["anchor_tags"].get_text().replace("/", "_")
        base_url = row["full_link"]
        main_topic = row["description"]
        if os.path.exists(f"{DATA_FOLDER}/{title}.json"):
            continue
        sections = html_extration_de(base_url)
        # save data to DataFrame
        df = pd.DataFrame(sections)
        # filtered data, empty chapter 
        df = df[df.content != ""]
        # change DataFrame to json
        json_data = {
            'main_topic': main_topic,
            'sections': df.to_dict(orient='records')
        }

        # save df to json
        import json
        if not os.path.exists(DATA_FOLDER):
            os.mkdir(DATA_FOLDER)
        with open(f"{DATA_FOLDER}/{title}.json", 'w') as f:
            json.dump(json_data, f, indent=4)

        # print current processing title
        tqdm.write(f"Processing {title}")

Getting link from subset A.
Processing to correct format.


Processing:   0%|          | 0/425 [00:00<?, ?it/s]

Processing  AABG 
Processing  AABGebV 
Processing  AAG 
Processing  AAppO 
Processing  AarhusÜbk 
Processing  AARKZustAnO 
Processing  AAÜbertrAnO 
Processing  AAÜG 
Processing  AAÜG-ÄndG 
Processing  AAÜGErstV 
Processing  AAustVorhAbk USA 
Processing  ABAG 
Processing  ABAMVerwV 
Processing  ABBergV 
Processing  ABBV 
Processing  AbfAEV 
Processing  AbfBeauftrV 
Processing  AbfKlärV 
Processing  AbfVerbrBußV 
Processing  AbfVerbrG 
Processing  AbgG 
Processing  AbgG10u11AbschnABestBek 
Processing  AbgrV 
Processing  Abk ISR 
Processing  AbrStV 
Processing  AbschlagsV 
Processing  AbsFondsForstAuflG 
Processing  AbsFondsLwAuflG 
Processing  ABV 
Processing  AbwAG 
Processing  AbwasserMeistPrV 
Processing  AbwUTechAusbV 
Processing  AbwV 
Processing  ABZusForstAbk POL 
Processing  ABZusForstAbkPOLG 
Processing  ABZusForstAbkPOLProt 
Processing  ABZusGrBrückVtr CES 
Processing  ABZusGrBrückVtrCESG 
Processing  ABZusGrBrückVtrCESNotw 
Processing  ABZusPolAbkG 
Processing  AdenauerHStiftG

Processing:   0%|          | 0/909 [00:00<?, ?it/s]

Processing  BAAZustV 
Processing  BAB-KAbgV 
Processing  BABauRaumOG 
Processing  BABefugV 2008 
Processing  BABG 
Processing  BABRiGeschwV 1978 
Processing  BABWismarG 
Processing  BadeV 
Processing  BADV 
Processing  BäAusbV 2004 
Processing  BäckMstrV 
Processing  BäderFAngAusbV 
Processing  BäderMeistPrV 
Processing  BÄO 
Processing  BÄOuaÄndG 2004 
Processing  BAFABGebV 
Processing  BAFG 
Processing  BAFinBefugV 
Processing  BaFinHwgebV 
Processing  BAFlSBAÜbnG 
Processing  BAföG 
Processing  BAföG-AuslandszuschlagsV 
Processing  BAföG-EinkommensV 
Processing  BAfög-FachlehrerV 
Processing  BAföG-MedPflegbV 
Processing  BAföG-TeilerlaßV 
Processing  BAföGZustV 2004 
Processing  BahnG 
Processing  BahnVorschrAnwG 
Processing  BAIUDBwOWiZustV 
Processing  BAKredSitzBek 
Processing  BAKredWWSUBefÜV 
Processing  BaktWaffVernÜbk 
Processing  BaktWaffVernÜbkG 
Processing  BALMÜV 
Processing  BAMBGebV 
Processing  BankFachwPrV 
Processing  BankkflAusbV 
Processing  BAPauschV 
Processing 

...

Processing  7. BinSchStrOAbweichV 
Processing  7. RAG 
Getting link from subset 8.
Processing to correct format.


Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing  8. GerüstbauerArbbV 
Processing  8. MARPOL-ÄndV 
Processing  8. RAG 
Getting link from subset 9.
Processing to correct format.


Processing:   0%|          | 0/5 [00:00<?, ?it/s]

Processing  9. BImSchV 
Processing  9. GebäudeArbbV 
Processing  9. ProdSV 
Processing  9. RAG 
Processing  9. RAV 


In [70]:
# remember to check the encoding before use the embedding model
text = """  "content": "(1) Das Gesetz tritt am Tag nach der Verk\u00fcndung in Kraft. (2)",
            "link": "https://www.gesetze-im-internet.de/aabg/BJNR068400002.htmlindex.html#BJNR068400002BJNE000400305"
"""
print(text)


  "content": "(1) Das Gesetz tritt am Tag nach der Verkündung in Kraft. (2)",
            "link": "https://www.gesetze-im-internet.de/aabg/BJNR068400002.htmlindex.html#BJNR068400002BJNE000400305"

